In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import pandas as pd
import numpy as np
from PIL import Image
import os
from os import walk

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/working/__notebook__.ipynb


In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies
%cd /content/yolov5
!pip install Pillow==9.0.0

Cloning into 'yolov5'...
remote: Enumerating objects: 12269, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 12269 (delta 5), reused 8 (delta 1), pack-reused 12250
Receiving objects: 100% (12269/12269), 12.02 MiB | 21.98 MiB/s, done.
Resolving deltas: 100% (8491/8491), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstallin

In [3]:
import pandas as pd
import numpy as np
from PIL import Image
from os import walk
import os

In [4]:
files = []
images = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename[-4:] == 'jpeg':
            #files += [os.path.join(dirname, filename)[:-5]]
            files += [filename[:-5]]
            #images += [np.array(Image.open(os.path.join(dirname, filename)))]
            dir_name = dirname
dir_name

'/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data'

In [5]:
os.mkdir("/kaggle/working/yolov5/datasets") 
os.mkdir("/kaggle/working/yolov5/datasets/test") 
os.mkdir("/kaggle/working/yolov5/datasets/train") 
os.mkdir("/kaggle/working/yolov5/datasets/valid") 
os.mkdir("/kaggle/working/yolov5/datasets/test/images") 
os.mkdir("/kaggle/working/yolov5/datasets/test/labels") 
os.mkdir("/kaggle/working/yolov5/datasets/train/images") 
os.mkdir("/kaggle/working/yolov5/datasets/train/labels") 
os.mkdir("/kaggle/working/yolov5/datasets/valid/images") 
os.mkdir("/kaggle/working/yolov5/datasets/valid/labels") 

In [6]:
import shutil
for i,file in enumerate(files):
    if i<len(files)*0.1:
        shutil.copyfile(dir_name+"/"+file+".jpeg", "/kaggle/working/yolov5/datasets/test/images/"+file+".jpeg")
        shutil.copyfile(dir_name+"/"+file+".txt", "/kaggle/working/yolov5/datasets/test/labels/"+file+".txt")
    elif i<len(files)*0.3:
        shutil.copyfile(dir_name+"/"+file+".jpeg", "/kaggle/working/yolov5/datasets/valid/images/"+file+".jpeg")
        shutil.copyfile(dir_name+"/"+file+".txt", "/kaggle/working/yolov5/datasets/valid/labels/"+file+".txt")
    else:
        shutil.copyfile(dir_name+"/"+file+".jpeg", "/kaggle/working/yolov5/datasets/train/images/"+file+".jpeg")
        shutil.copyfile(dir_name+"/"+file+".txt", "/kaggle/working/yolov5/datasets/train/labels/"+file+".txt")

In [7]:
outF = open("/kaggle/working/data.yaml", "w")
outF.write("names:\n- 'crop'\n- 'weed'\nnc: 2\ntrain: datasets/train/images\nval: datasets/valid/images")
outF.close()

In [8]:
import torch
from IPython.display import Image  # for displaying images
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.11.0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [9]:
!python /kaggle/working/yolov5/train.py --img 512 --cfg /kaggle/working/yolov5/models/yolov5s.yaml --batch 4 --epochs 15 --data /kaggle/working/data.yaml --weights yolov5s.pt --workers 24 --name yolo_road_det

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=/kaggle/working/yolov5/models/yolov5s.yaml, data=/kaggle/working/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=15, batch_size=4, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=yolov5/runs/train, name=yolo_road_det, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-259-ga2a1ed2 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-1

In [10]:
!python /kaggle/working/yolov5/detect.py --conf 0.4  --source /kaggle/working/yolov5/datasets/test/images

detect: weights=yolov5/yolov5s.pt, source=/kaggle/working/yolov5/datasets/test/images, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-259-ga2a1ed2 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

100%|██████████████████████████████████████| 14.1M/14.1M [00:00<00:00, 48.3MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
image 1/130 /kaggle/working/yolov5/datasets/test/images/agri_0_1028.jpeg: 640x640 Done. (0.008s)
image 2/130 /kaggle/working/yolov5/datasets/test/images/agri_0_1130.jpeg: 640x640 Done. (0.007s)
image 3/130 /kaggle/working/yolov5/datasets/test/images/agri_0_1140.j